# FAST-UAV - Multirotor Design Optimization
*Author: Félix Pollet - 2023* <br>

This notebook is basically the same as the [Tutorial Notebook](0_Tutorial.ipynb), but it is more concise so that you can quickly re-use it to find the optimum sizing for any multirotor UAV specification.

## 1. Setting up and analyzing the problem

Now that you are familiar with FAST-UAV, you may define your own design problem. In the following cell, feel free to use your own source file (initially, [problem_inputs_DJI_M600.xml](./data/source_files/problem_inputs_DJI_M600.xml)) and configuration file (initially, [multirotor_mdo.yaml](./data/configurations/multirotor_mdo.yaml)). Remember that the configuration file defines the design problem, i.e. the model, the optimization problem definition and the optimization algorithm, while the source file is a reference file for the problem's input values.

In [ ]:
# Import required librairies
import os.path as pth
import sys
import logging
import fastoad.api as oad
import shutil
from fastuav.utils.postprocessing.analysis_and_plots import multirotor_geometry_plot, mass_breakdown_sun_plot_drone

sys.path.append(pth.abspath("."))

#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.display import display, HTML, IFrame
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Paths to folders and files
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
CONFIGURATION_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "configurations")
SOURCE_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "source_files")

CONFIGURATION_FILE = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_mdo.yaml")  # You may provide a different configuration file
SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_inputs_DJI_M600.xml")  # You may provide a different source file

Let's have a look at the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

Now we can generate the temporary input file `workdir/problem_inputs.xml` from the source file you have specified earlier:

In [ ]:
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

You can now checkout the generated [input file](./workdir/problem_inputs.xml). The values in this file can be modified by the user (e.g., if the values specified in the original source file are uncorrect or missing). These changes will remain in the scope of this notebook. That is, it will not affect the source file, and the changes will disappear when you leave the session.

In [ ]:
oad.variable_viewer(input_file)

## 2. Running the MDO

You can now run an optimization problem. Remember that the last part of the configuration file .yaml is where this optimization problem is defined. For example, you may change the objective function to maximize the range instead of minimizing the mtow.

```yaml
optimization:
  design_variables:
    - name: data:weights:mtow:k # over estimation coefficient on the load mass
      upper: 40.0
      lower: 1.0
  constraints:
    - name: optimization:constraints:weights:mtow:guess # mass consistency
      lower: 0.0
  objective:
    - name: data:weights:mtow
      scaler: 1e-1
```

In [ ]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

Let's save these results permanently:

In [ ]:
output_file = optim_problem.output_file_path
shutil.copy(output_file, 
            pth.join(SOURCE_FOLDER_PATH, 'problem_outputs_DJI_M600_mdo.xml')
)

The `optimizer_viewer` offers a convenient summary of the optimization result. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated.

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
oad.variable_viewer(output_file)

## 3. Analysis and plots

You can now use postprocessing plots to visualize the results of the MDO.

In [ ]:
fig = multirotor_geometry_plot(output_file)
fig.show()

In [ ]:
fig = mass_breakdown_sun_plot_drone(output_file)
fig.show()